In [ ]:
!pip install pytrec_eval
!pip install rank_bm25
!pip install ast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached AST-0.0.2.tar.gz (19 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!git clone https://github.com/aliannejadi/ClariQ

fatal: destination path 'ClariQ' already exists and is not an empty directory.


In [ ]:
import pandas as pd
from statistics import mean
import os 
import logging
import os
import sys
import random
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
  stemmer = PorterStemmer()
  tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
  tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
  return [stemmer.stem(word) for word in tokens]

np.random.seed(42)
random.seed(42)


logging.basicConfig(
  level=logging.INFO,
  format="%(asctime)s [%(levelname)s] %(message)s",
  handlers=[
      logging.StreamHandler(sys.stdout)
  ]
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.model_selection import train_test_split
#selecting part of data for dev set for testing
datapath = '/content/ClariQ/data/dev.tsv'
df_dev = pd.read_csv(datapath, sep='\t')

dev = df_dev.drop(columns=['clarification_need'])
dev = dev[~dev["question"].isnull()]

#preprocessing question bank for BM25
question_bank_path = '/content/ClariQ/data/question_bank.tsv'
question_bank = pd.read_csv(question_bank_path,sep='\t').fillna('')
question_bank['tokenized_question_list'] = question_bank['question'].map(stem_tokenize)
question_bank['tokenized_question_str'] = question_bank['tokenized_question_list'].map(lambda x: ' '.join(x))

In [ ]:
run_file_path = 'dev_bm25'

bm25_corpus = question_bank['tokenized_question_list'].tolist()
bm25 = BM25Okapi(bm25_corpus)
rerank_top_k = 30
with open(run_file_path, 'w') as fo:
  for tid in df_dev['topic_id'].unique():
    query = df_dev.loc[df_dev['topic_id']==tid, 'initial_request'].tolist()[0]
    bm25_ranked_list = bm25.get_top_n(stem_tokenize(query, True), 
                                    bm25_corpus, 
                                    n=30)
    bm25_q_list = [' '.join(sent) for sent in bm25_ranked_list]
    docs = question_bank.set_index('tokenized_question_str').loc[bm25_q_list, 'question'].tolist()
    preds = question_bank.set_index('tokenized_question_str').loc[bm25_q_list, 'question_id'].tolist()
   
    for i, qid in enumerate(preds):    
      fo.write('{} 0 {} {} {} bm25\n'.format(tid, qid, i, len(preds)-i))

In [ ]:
    topic_df = df_dev
    run_file = run_file_path
    topic_question_set_dict = topic_df.groupby('topic_id')['question_id'].agg(set).to_dict()
    run_df = pd.read_csv(run_file, sep=' ', header=None)
    run_df = run_df.sort_values(by=[0, 4], ascending=False).drop_duplicates(subset=[0, 4], keep='first')
    run_question_set_list = run_df.groupby(0)[2].agg(list).to_dict()
    topk_list = [5, 10, 20, 30]
    recall_score_dict = {}
    for topk in topk_list:
        metric_name = 'Recall{}'.format(topk)
        recall_score_dict[metric_name] = {}
        for tid in topic_question_set_dict:
            rec = len(set(run_question_set_list[tid][:topk]) & topic_question_set_dict[tid]) / len(
                    topic_question_set_dict[tid])
            recall_score_dict[metric_name][tid] = rec


    mean_performance = {}
    for metric in recall_score_dict:
        mean_performance[metric] = mean(recall_score_dict[metric][k] for k in recall_score_dict[metric])

    for metric in recall_score_dict:
        print('{}: {}'.format(metric, mean_performance[metric]))

Recall5: 0.3245570421150917
Recall10: 0.5638042646208281
Recall20: 0.6674997108155003
Recall30: 0.6912818698329535


In [ ]:
  import pickle
  topic_df = df_dev
  # print(topic_df)
  context_array = topic_df['facet_id'].values

  eval_file_path = '/content/ClariQ/data/single_turn_train_eval.pkl'
  with open(eval_file_path, 'rb') as fi:
            eval_dict = pickle.load(fi)

  new_eval_dict = {}
  for metric in eval_dict:
      new_eval_dict[metric] = {}
      for fid in eval_dict[metric]:
          if fid in context_array:
                new_eval_dict[metric][fid] = eval_dict[metric][fid]

  eval_dict = new_eval_dict
  
  run_df = pd.read_csv(run_file, sep=' ', header=None).fillna('')
  run_df = run_df.sort_values(by=4).drop_duplicates(subset=[0], keep='last')  # we only keep the top ranked question.
  run_dict = run_df.set_index(0)[2].to_dict()
  
  facet_to_topic_dict = topic_df.set_index('facet_id')['topic_id'].to_dict()

  performance_dict = {}
  for metric in eval_dict:
        performance_dict[metric] = {}
        for context_id in eval_dict[metric]:
            try:
              selected_q = run_dict[facet_to_topic_dict[context_id]]
              selected_q = 'MIN' if selected_q == 'MAX' else selected_q 
              
              try:
                performance_dict[metric][context_id] = eval_dict[metric][context_id][selected_q]['with_answer']
              except KeyError:  # if question is not among candidate question, we consider it equal to minimum performance.
                performance_dict[metric][context_id] = eval_dict[metric][context_id]['MIN']['with_answer']
            except KeyError:  # if there is no prediction provided for a facet, we consider performance 0.
              performance_dict[metric][context_id] = 0.
  

    # compute the mean performance per metric and print
  mean_performance = {}
  for metric in performance_dict:
        mean_performance[metric] = mean(performance_dict[metric][k] for k in performance_dict[metric])

In [ ]:
 for metric in performance_dict:
            print('{}: {}'.format(metric, mean_performance[metric]))

NDCG1: 0.1859375
NDCG3: 0.16076878817226317
NDCG5: 0.15299655291242895
NDCG10: 0.13569348827493657
NDCG20: 0.12814916594090767
P1: 0.23125
P3: 0.18958333333333333
P5: 0.17500000000000002
P10: 0.139375
P20: 0.1178125
MRR100: 0.3090313834647762
